# Kubeflow Serving (KFServing) 

This is a sample for KFServing SDK. 

The notebook shows how to use KFServing SDK to create, get, rollout_canary, promote and delete InferenceService.

In [ ]:
!pip install kfserving==0.3.0.1 --user

In [ ]:
# Restart the kernel to pick up pip installed libraries
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import time
time.sleep(5)

from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements

Define namespace where InferenceService needs to be deployed to. If not specified, below function defines namespace to the current one where SDK is running in the cluster, otherwise it will deploy to default namespace.

In [ ]:
namespace = utils.get_default_target_namespace()
print(namespace)

### Allow KFServing Inference Services to be created from the notebook

Adding this on ClusterRole to skip error: inferenceservices.serving.kubeflow.org is forbidden: User system:serviceaccount:anonymous:default-editor cannot create resource inferenceservices

```shell
kubectl edit clusterrole kubeflow-kubernetes-edit -n kubeflow
```

Add following policies to cluster role lists.
```yaml
- apiGroups:
  - kubeflow.org
  - serving.kubeflow.org
  resources:
  - '*'
  verbs:
  - '*'
- apiGroups:
  - networking.istio.io
  resources:
  - '*'
  verbs:
  - '*'
```

## Define InferenceService

Firstly define default endpoint spec, and then define the inferenceservice basic on the endpoint spec.

In [ ]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            min_replicas=1,
                            tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='gs://kfserving-samples/models/tensorflow/flowers',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'0.5Gi'},
                                  limits={'cpu':'100m', 'memory':'0.5Gi'}))))
    
isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='flowers-sample', namespace=namespace),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))

## Create InferenceService to Receive 100% Traffic

Call KFServingClient to create InferenceService.

In [ ]:
kf_serving = KFServingClient()
kf_serving.create(isvc)

### Check the Status of InferenceService
_Note:  Re-run this until the service shows READY=True_

In [ ]:
!kubectl get inferenceservices -n $namespace

In [ ]:
kf_serving.get('flowers-sample', 
               namespace=namespace, 
               watch=True, 
               timeout_seconds=120)

In [ ]:
!kubectl -n anonymous get inferenceservice flowers-sample -o jsonpath='{.status.url}' | cut -d "/" -f 3

## Run a Prediction

In [ ]:
%%bash

INPUT_PATH=@./input.json

MODEL_NAME=flowers-sample
echo $MODEL_NAME

INGRESS_GATEWAY=kfserving-ingressgateway
echo $INGRESS_GATEWAY

ISTIO_HOST_IP=$(kubectl get po -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].status.hostIP}')
echo $ISTIO_HOST_IP

ISTIO_NODE_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath="{.spec.ports[?(@.name=='http2')].nodePort}")
echo $ISTIO_NODE_PORT

SERVICE_HOSTNAME=$(kubectl -n anonymous get inferenceservice ${MODEL_NAME} -o jsonpath='{.status.url}' | cut -d "/" -f 3)
echo $SERVICE_HOSTNAME

curl -v -H "Host: ${SERVICE_HOSTNAME}"  http://${ISTIO_HOST_IP}:${ISTIO_NODE_PORT}/v1/models/${MODEL_NAME}:predict -d $INPUT_PATH


In [ ]:
%%bash
INPUT_PATH=@./input.json

MODEL_NAME=flowers-sample
echo $MODEL_NAME

SERVICE_HOSTNAME=$(kubectl -n anonymous get inferenceservice ${MODEL_NAME} -o jsonpath='{.status.url}' | cut -d "/" -f 3)
echo $SERVICE_HOSTNAME

curl -v -H "Host: ${SERVICE_HOSTNAME}" http://${SERVICE_HOSTNAME}/v1/models/${MODEL_NAME}:predict -d $INPUT_PATH

In [ ]:
%%bash

MODEL_NAME=sklearn-iris
echo $MODEL_NAME

SERVICE_HOSTNAME=$(kubectl -n anonymous get inferenceservice ${MODEL_NAME} -o jsonpath='{.status.url}' | cut -d "/" -f 3)
echo $SERVICE_HOSTNAME

curl -v -H "Host: sklearn-iris.anonymous.example.com" http://sklearn-iris.anonymous.example.com/v1/models/sklearn-iris:predict -d '{"instances":[[6.8,2.8,4.8,1.4],[6.0,3.4,4.5,1.6]]}'


In [ ]:
!kubectl -n anonymous get inferenceservice sklearn-iris -o jsonpath='{.status.url}' | cut -d "/" -f 3

In [ ]:
!kubectl  get gateway kfserving-gateway

In [ ]:
!kubectl -n istio-system get service $INGRESS_GATEWAY

In [ ]:
%%bash

MODEL_NAME=flowers-sample
INPUT_PATH=@./input.json
INGRESS_GATEWAY=kfserving-ingressgateway

CLUSTER_IP=$(kubectl -n istio-system get service $INGRESS_GATEWAY -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
echo "ClusterIP: $CLUSTER_IP"

SERVICE_HOSTNAME=$(kubectl get inferenceservice ${MODEL_NAME} -o jsonpath='{.status.url}' | cut -d "/" -f 3)
echo $SERVICE_HOSTNAME

curl -v -H "Host: ${SERVICE_HOSTNAME}" http://$CLUSTER_IP/v1/models/$MODEL_NAME:predict -d $INPUT_PATH

In [ ]:
!kubectl -n istio-system get svc istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}'

## Add a New InferenceService (with a New Model) to Receive 10% Traffic
Firstly define canary endpoint spec, and then rollout 10% traffic to the canary version, watch the rollout process.

In [ ]:
canary_endpoint_spec = V1alpha2EndpointSpec(
                         predictor=V1alpha2PredictorSpec(
                           min_replicas=1,
                           tensorflow=V1alpha2TensorflowSpec(
                             storage_uri='gs://kfserving-samples/models/tensorflow/flowers-2',
                             resources=V1ResourceRequirements(
                                 requests={'cpu':'100m','memory':'0.5Gi'},
                                 limits={'cpu':'100m', 'memory':'0.5Gi'}))))

In [ ]:
kf_serving.rollout_canary('flowers-sample', 
                          canary=canary_endpoint_spec, 
                          percent=10,
                          namespace=namespace, 
                          watch=True, 
                          timeout_seconds=120)

In [ ]:
!kubectl get inferenceservices -n $namespace

In [ ]:
%%bash
MODEL_NAME=flowers-sample
INPUT_PATH=@./input.json
INGRESS_GATEWAY=istio-ingressgateway

ISTIO_HOST_IP=$(kubectl get po -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].status.hostIP}')
echo $ISTIO_HOST_IP

ISTIO_NODE_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath="{.spec.ports[?(@.name=='http2')].nodePort}")
echo $ISTIO_NODE_PORT

SERVICE_HOSTNAME=$(kubectl -n anonymous get inferenceservice ${MODEL_NAME} -o jsonpath='{.status.url}' | cut -d "/" -f 3)

curl -v -H "Host: ${SERVICE_HOSTNAME}"  http://${ISTIO_HOST_IP}:${ISTIO_NODE_PORT}/v1/models/${MODEL_NAME}:predict -d $INPUT_PATH


## Send More Traffic to the New InferenceService
Send 50% traffic to the new model

In [ ]:
kf_serving.rollout_canary('flowers-sample',
                          percent=50, 
                          namespace=namespace,
                          watch=True, 
                          timeout_seconds=120)

In [ ]:
!kubectl get inferenceservices -n $namespace

In [ ]:
%%bash
MODEL_NAME=flowers-sample
INPUT_PATH=@./input.json
INGRESS_GATEWAY=istio-ingressgateway

ISTIO_HOST_IP=$(kubectl get po -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].status.hostIP}')
echo $ISTIO_HOST_IP

ISTIO_NODE_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath="{.spec.ports[?(@.name=='http2')].nodePort}")
echo $ISTIO_NODE_PORT

SERVICE_HOSTNAME=$(kubectl -n anonymous get inferenceservice ${MODEL_NAME} -o jsonpath='{.status.url}' | cut -d "/" -f 3)

curl -v -H "Host: ${SERVICE_HOSTNAME}"  http://${ISTIO_HOST_IP}:${ISTIO_NODE_PORT}/v1/models/${MODEL_NAME}:predict -d $INPUT_PATH


## (Optional) Delete the InferenceService

In [ ]:
kf_serving.delete('flowers-sample', 
                  namespace=namespace)